In [ ]:
import fiber
from fiber.cohort import Cohort
from fiber.condition import Patient, MRNs
from fiber.condition import Diagnosis
from fiber.condition import Measurement, Encounter, Drug, TobaccoUse,LabValue
from fiber.storage import yaml as fiberyaml
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import os
import matplotlib.pyplot as plt
from functools import reduce
import json
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import category_encoders as ce

In [ ]:
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import os

In [ ]:
# All unsupervised with 0.1 threshold for Proc, Lab, VS and 0.05 for Diagnosis and Drug
Diagnosis = pd.read_pickle('Clean_Unsupervised_Dataframes/Diagnosis_Cleaned_no_int.pkl')
Drug = pd.read_pickle('Clean_Unsupervised_Dataframes/Drugs_Cleaned.pkl')
Proc = pd.read_pickle('Clean_Unsupervised_Dataframes/Procedures_Cleaned.pkl')
VS = pd.read_pickle('Clean_Unsupervised_Dataframes/VitalSigns_Cleaned.pkl')
Lab = pd.read_pickle('Clean_Unsupervised_Dataframes/LabValues_Cleaned.pkl')

In [ ]:
df_final_extraBP = pd.read_pickle('/home/kiwitn01/master_thesis_hypertension-complications/Feature Extraction/Clean_Unsupervised_Dataframes/All_unsupervised_merged_clean_Diag_no_int_with_extra_BP.pkl')
df_final = pd.read_pickle('/home/kiwitn01/master_thesis_hypertension-complications/Feature Extraction/Clean_Unsupervised_Dataframes/All_unsupervised_merged_clean_Diag_no_int.pkl')




In [ ]:
for col in df_final:
    print (col)

In [ ]:
Proc_check = [col for col in df_final if col.startswith('Drug')]

In [ ]:
Proc_check

In [ ]:
df_final[df_final.columns[pd.Series(df_final.columns).str.startswith('Diagnosis')]]

In [ ]:
#pip install category_encoders

In [ ]:
#This Notebook can be used to clean the unsupervised extracted features.
#For numerical features(Lab and Vital Sign) It is dropping features were a certain threshold of patients 
#do not have this features and is replacing outlier values with NaN so that they can be later imputed 

#Furthermore are measures for the same type of test out of different Systems like IBEX or EPIC are dropped.
#For Categorical Features(Medication Diagnosis and procedures) the user can choose which further features should be dropped,
#by entering the feature name.



In [ ]:
# Quanitle function

# dropping numeric values if over 95% of the patients have nan values 
# replacing outliers with NaN values so that they can be imputed afterwards

def cleanNumFeatures(df,featurename): 
    feature=featurename
    initial_nan=df[feature].isna().sum()
    if initial_nan>55000:
        return False 
    print(initial_nan)
    test=df 
    test=test[feature].dropna()
    upper_quantile=np.quantile(test.values,0.995,axis=0)
    lower_quantile=np.quantile(test.values,0.005,axis=0)
    df.loc[df[feature]>=upper_quantile,[feature]]=np.nan
    df.loc[df[feature]<=lower_quantile,[feature]]=np.nan
    if df[feature].isna().sum()>=55000: 
        return False
    return True

col_for_dropping=['age_in_days',
'date_of_birth',
'month_of_birth',
'gender',
'religion',
'race',
'patient_ethnic_group',
'deceased_indicator',
'mother_account_number',
'address_zip',
'marital_status_code','HT']

In [ ]:
# All unsupervised with 0.1 threshold for Proc, Lab, VS and 0.05 for Diagnosis and Drug
Diagnosis = pq.read_table('Parquets/Cases/Consolidated/with_0.05_threshold/Diagnosis_Cases_Unsupervised_All.parquet').to_pandas()
Drug = pq.read_table('Parquets/Cases/Consolidated/with_0.05_threshold/Drug_Cases_Unsupervised_All.parquet').to_pandas()
Proc = pq.read_table('Parquets/Cases/Consolidated/with_0.1_threshold/Procedure_Cases_Unsupervised_All.parquet').to_pandas()
VS = pq.read_table('Parquets/Cases/Consolidated/with_0.1_threshold/VitalSign_Cases_Unsupervised_All.parquet').to_pandas()
Lab = pq.read_table('Parquets/Cases/Consolidated/with_0.1_threshold/LabValue_Cases_Unsupervised_All.parquet').to_pandas()


In [ ]:
Diagnosis = pq.read_table('Parquets/Cases/Consolidated/with_0.05_threshold/Diagnosis_Cases_Unsupervised_All.parquet').to_pandas()


In [ ]:
Diagnosis

In [ ]:
VS

In [ ]:
#LAB VALUES
lab_df = Lab
lab_df = lab_df.reset_index('medical_record_number')

#drop mrns which will be later added again
further_col_drop = []
mrn = pd.DataFrame()
mrn['medical_record_number'] = lab_df['medical_record_number']
lab_df = lab_df.drop('medical_record_number',axis=1)
lab_df = lab_df.drop(col_for_dropping,axis=1)
lab_df

#
col_names = lab_df.columns
for c in col_names: 
    print(c)
    keep_column = cleanNumFeatures(lab_df,c)
    if keep_column == False:
        #lab_df=lab_df.drop(c,axis=1)
        further_col_drop.append(c)
        print('drop')
    #print(lab_df)
    
lab_df= lab_df.drop(further_col_drop, axis=1)
lab_df['medical_record_number'] = mrn['medical_record_number']

In [ ]:
lab_df

In [ ]:
#save as pkl
#lab_df.to_pickle("Clean_Unsupervised_Dataframes/LabValues_Cleaned.pkl")

In [ ]:
#count and plot NaN values -> number of columns = x-axis, amount of patients missing the values = y-axis
d=lab_df.isnull().sum(axis=1) 

import matplotlib.pyplot as plt
plt.hist(x=d, bins=137, color='#0504aa',alpha=0.7, rwidth=0.85)

In [ ]:
lab_df['count_missing']=d
lab_df

In [ ]:
lab_df.loc[lab_df['count_missing'] <= 100]

In [ ]:
#Vital Signs

In [ ]:
vital_df = VS
vital_df = vital_df.reset_index('medical_record_number')

In [ ]:
further_col_drop=[]
mrn=pd.DataFrame()
mrn['medical_record_number']=vital_df['medical_record_number']
vital_df=vital_df.drop('medical_record_number',axis=1)
vital_df=vital_df.drop(col_for_dropping,axis=1)
vital_df
col_names=vital_df.columns
for c in col_names: 
    print(c)
    keep_column=cleanNumFeatures(vital_df,c)
    if keep_column==False:
        #lab_df=lab_df.drop(c,axis=1)
        further_col_drop.append(c)
        print('drop')
    #print(lab_df)
vital_df=vital_df.drop(further_col_drop,axis=1)
vital_df['medical_record_number']=mrn['medical_record_number']

In [ ]:
vital_df

In [ ]:
list(vital_df.columns)

In [ ]:
#keep only the EPIC values since all values out of the different values are the same
further_col_drop=[
 'max__VitalSign__IBEX__(RESP)',
 'max__VitalSign__IBEX__(T-O)',
 'max__VitalSign__IBEX__(T-T)',
 'max__VitalSign__IBEX__DBP',
 'max__VitalSign__IBEX__PULSE',
 'max__VitalSign__IBEX__PULSE OXIMETRY',
 'max__VitalSign__IBEX__SBP',
 'max__VitalSign__IBEX__TEMPERATURE',
 'max__VitalSign__TDS__(RESP)',
 'max__VitalSign__TDS__(T-O)',
 'max__VitalSign__TDS__(T-T)', 
 'max__VitalSign__TDS__DBP',
 'max__VitalSign__TDS__SBP',
 'median__VitalSign__IBEX__(RESP)',
 'median__VitalSign__IBEX__(T-O)',
 'median__VitalSign__IBEX__(T-T)',
 'median__VitalSign__IBEX__DBP',
 'median__VitalSign__IBEX__PULSE',
 'median__VitalSign__IBEX__PULSE OXIMETRY',
 'median__VitalSign__IBEX__SBP',
 'median__VitalSign__IBEX__TEMPERATURE',
 'median__VitalSign__TDS__(RESP)',
 'median__VitalSign__TDS__(T-O)',
 'median__VitalSign__TDS__(T-T)',
 'median__VitalSign__TDS__DBP',
 'median__VitalSign__TDS__SBP',
 'min__VitalSign__IBEX__(RESP)',
 'min__VitalSign__IBEX__(T-O)',
 'min__VitalSign__IBEX__(T-T)',
 'min__VitalSign__IBEX__DBP',
 'min__VitalSign__IBEX__PULSE',
 'min__VitalSign__IBEX__PULSE OXIMETRY',
 'min__VitalSign__IBEX__SBP',
 'min__VitalSign__IBEX__TEMPERATURE',
 'min__VitalSign__TDS__(RESP)',
 'min__VitalSign__TDS__(T-O)',
 'min__VitalSign__TDS__(T-T)',
 'min__VitalSign__TDS__DBP',
 'min__VitalSign__TDS__SBP'
]

In [ ]:
vital_df = vital_df.drop(further_col_drop,axis=1)


In [ ]:
# check DBP, lower than 50, bigger than 120 
vital_df.loc[vital_df['max__VitalSign__EPIC__DBP'] < 50]

In [ ]:
#save VS
#vital_df.to_pickle("Clean_Unsupervised_Dataframes/VitalSigns_Cleaned.pkl")

In [ ]:
# Procedures

In [ ]:
proc_df = Proc
proc_df = proc_df.reset_index('medical_record_number')

In [ ]:
proc_df 

In [ ]:
further_col_drop=[]

for c in proc_df.columns: 
    if ('EPIC' not in c) : 
        #print(c)
        further_col_drop.append(c)
        
further_col_drop

In [ ]:
further_col_drop = [
    'HT',
 'age_in_days',
 'date_of_birth',
 'month_of_birth',
 'gender',
 'religion',
 'race',
 'patient_ethnic_group',
 'deceased_indicator',
 'mother_account_number',
 'address_zip',
 'marital_status_code',
 'Procedure__SYSTEM__MSDW_NOT APPLICABLE',
 'Procedure__SYSTEM__MSDW_UNKNOWN',
 'Procedure__TDS__(RESP)',
 'Procedure__TDS__(T-O)',
 'Procedure__TDS__(T-T)', 
 'Procedure__TDS__DBP',
 'Procedure__TDS__SBP',
 'Procedure__TDS__PAIN SCORE',
 'Procedure__IBEX__WEIGHT',
]

In [ ]:
proc_df=proc_df.drop(further_col_drop,axis=1) 

In [ ]:
proc_df

In [ ]:
#proc_df.to_pickle("Clean_Unsupervised_Dataframes/Procedures_Cleaned.pkl")

In [ ]:
#Diagnosis

In [ ]:
Diagnosis

In [ ]:
list(Diagnosis)

In [ ]:
diag_df = Diagnosis

In [ ]:
further_col_drop =[
 'Diagnosis__SYSTEM__MSDW_NOT APPLICABLE',
 'Diagnosis__SYSTEM__MSDW_UNKNOWN',
 'HT',
 'age_in_days',
 'date_of_birth',
 'month_of_birth',
 'gender',
 'religion',
 'race',
 'patient_ethnic_group',
 'deceased_indicator',
 'mother_account_number',
 'address_zip',
 'marital_status_code',
 'Diagnosis__EPIC__FOLLOW UP',
 'Diagnosis__ICD-10__I10',
    'Diagnosis__ICD-9__401.1',
    'Diagnosis__ICD-9__401.9',
    'Diagnosis__IMO__158530',
    'Diagnosis__IMO__154288'
]

In [ ]:
diag_df = diag_df.drop(further_col_drop,axis=1)

In [ ]:
diag_df

In [ ]:
# transfer Diag to int
diag_df_int = diag_df * 1

In [ ]:
diag_df_int

In [ ]:
#diag_df.to_pickle("Clean_Unsupervised_Dataframes/Diagnosis_Cleaned_no_int.pkl")

In [ ]:
#Drugs

In [ ]:
Drug

In [ ]:
list(Drug)

In [ ]:
further_col_drop = ['HT',
 'age_in_days',
 'date_of_birth',
 'month_of_birth',
 'gender',
 'religion',
 'race',
 'patient_ethnic_group',
 'deceased_indicator',
 'mother_account_number',
 'address_zip',
 'marital_status_code'
]

In [ ]:
drug_df = Drug

In [ ]:
drug_df = drug_df.drop(further_col_drop,axis=1)

In [ ]:
drug_df

In [ ]:
#drug_df.to_pickle("Clean_Unsupervised_Dataframes/Drugs_Cleaned.pkl")